In [1]:
# !pip install ragas langchain langchain-openai

In [2]:
import pandas as pd

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from ragas import evaluate
from ragas.testset import TestsetGenerator
from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from src.utils.config import ConfigDB, ConfigLLM
from src.agent.prompts.analyst_prompt import ANALYSIS_SYSTEM_PROMPT




/Users/kim/Documents/lucyMacBookPro/SKN/팀프로젝트_4차/SKN21-4th-4Team/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_5302/529838623.py:13: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_5302/529838623.py:13: DeprecationWarning: Importing LLMContextPrecisionWithReference from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metri

In [3]:
COLLECTION_NAME = ConfigDB.COLLECTION_NAME
EMBEDDING_MODEL = ConfigDB.EMBEDDING_MODEL
VECTOR_SIZE = ConfigDB.VECTOR_SIZE
OPENAI_MODEL = ConfigLLM.OPENAI_MODEL

COLLECTION_NAME, EMBEDDING_MODEL, VECTOR_SIZE, OPENAI_MODEL

('learning_ai', 'text-embedding-3-large', 3072, 'gpt-4o-mini')

In [4]:
# 데이터셋을 생성할 때 사용할 context를 추출 - sampling
client = QdrantClient(host=ConfigDB.HOST, port=ConfigDB.PORT)

# 전체 데이터를 다 조회해서 그 중 랜덤하게 K개만 sampling
info = client.get_collection(COLLECTION_NAME)
total_count = info.points_count

print("info : ", info)
print("total_count : ", total_count)

ResponseHandlingException: [Errno 61] Connection refused

In [ ]:
results, next_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=info.points_count,
)

# sampling
import random
sample_dataset = random.sample(results, 5) # 리스트에서 랜덤하게 K개를 추출


# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset]
docs

In [ ]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

def get_vectorstore(collection_name: str):
    # 1. 임베딩 모델 설정
    dense_embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    # 2. Qdrant 클라이언트 연결
    client = QdrantClient(host="localhost", port=6333)
    # 3. 기존 컬렉션에 연결
    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=dense_embeddings
    )
    
    return vectorstore


def get_retriever(vectorstore, k: int = 5):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [ ]:
vectorstore = get_vectorstore(COLLECTION_NAME)
retriever = get_retriever(vectorstore)

In [ ]:
################################################################################
# 평가할 RAG Chain
################################################################################
prompt_txt = ANALYSIS_SYSTEM_PROMPT
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
)


def format_docs(documents:list)->str:
    """
    VectorStore에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    VectorStore의 검색결과인 List[Document]를 받아서 Document들에서 page_content의 내용만 추출한다.
    
    Args:
        documents(list[Document]): [Document(..), Document(...), ..]}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    return "\n\n".join(doc.page_content for doc in documents)

model = ChatOpenAI(model=OPENAI_MODEL)
parser = StrOutputParser()

# RAG 평가를 위해서 "답변", "검색한 문서" 둘이 출력되도록 변경.
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

def format_doc_list(docs_dict: dict) -> list:
    # dictionary[list[Document], query:str] -> list[str]  
    # 문서내용만 추출해서(Document.page_content)만 추출한 리스트
    return [doc.page_content for doc in docs_dict['context']]

# dict | dict -> dict
# RunnablePassthrough() | dict | dict ==> RunnableSequence
chain = RunnablePassthrough() | {
    "context":retriever,
    "query":RunnablePassthrough()
} | {
    "response": prompt | model | parser,
    "retrieved_context": format_doc_list, # RAGAS 평가시 context -> List[str]
}

In [ ]:
res = chain.invoke("파인튜닝의 장점은 무엇인가?")
res

In [ ]:
llm_context = """당신은 Python 프로그래밍 교육 과정의 **AI 조교(Teaching Assistant)**이자 데이터셋 생성 전문가입니다.
제공된 [강의 자료(Lecture)]와 [Python 공식 문서(RST)]를 기반으로, 학습자 평가를 위한 고품질의 JSON 포맷 질의응답(QA) 데이터셋을 생성하십시오.

생성 시 다음의 **지침(Instruction)과 답변 스타일(Tone & Style)**을 철저히 준수해야 합니다.

---

## Instruction
당신은 실제 강의를 보조하는 **AI 조교(Teaching Assistant) Agent**이다.  
search agent가 제공한 **강의 자료(context)** 를 바탕으로,  
수강생의 질문에 대해 **강의 흐름 안에서 이해를 돕는 답변**을 제공한다.

---

## 기본 원칙

1. **질문 중심**
   - 사용자의 질문 의도를 먼저 파악하고, 그 질문에 직접적으로 필요한 내용만 답변한다.
   - 질문에 포함되지 않은 개념 확장이나 일반적인 배경 설명은 하지 않는다.

2. **강의 맥락 우선**
   - 모든 설명은 제공된 강의 자료(context)의 내용과 흐름을 기준으로 한다.
   - 내부 강의 자료([Original])를 최우선으로 사용하며, 외부 자료([External Web])가 있는 경우 보조적으로만 참고한다.

3. **불필요한 정보 배제**
   - 질문과 직접적인 관련이 없는 내용은 포함하지 않는다.
   - 설명이 가능하더라도 질문 범위를 벗어난다면 생략한다.

4. **정직한 한계 표현**
   - 질문에 대한 직접적인 근거가 강의 자료에 없는 경우,
     “강의 자료에서는 해당 내용이 직접적으로 다뤄지지 않는다”고 명확히 말한다.
   - 다만, 강의 흐름상 최소한의 맥락 설명이 필요한 경우에만 짧게 보충한다.

---

## 답변 방식

- 먼저 **질문에 대한 핵심 답**을 간단히 제시한다.
- 이후 필요할 경우에만:
  - 강의 자료의 어느 부분과 연결되는지
  - 코드나 실습이 있다면 그 목적과 역할
  을 설명한다.
- 질문이 단순한 경우에는 추가 설명 없이 핵심 답변만 제공해도 된다.

---

## Output Guide (유연 적용)

필요한 경우에만 아래 요소를 포함한다:

- **핵심 답변**: 질문에 대한 직접 관련된 핵심 개념을 간단히 제시한다.
- **강의 맥락 설명**: 강의 자료에서 어떤 파일 / 어떤 코드 또는 셀과 연결되는지 명확히 설명한다.
- **실습/코드 연결**: 왜 이 코드가 등장했는지에 대한 설명을 수업 흐름 기준으로 설명한다.
- **한 줄 정리**: 시험 대비 또는 복습용으로 한 문장으로 요약한다.

모든 요소를 항상 포함할 필요는 없다.

---

## Tone & Style

- 실제 강의 조교처럼 **차분하고 설명 중심적인 말투**를 사용한다.
- 과장, 불필요한 비유, 감정 표현은 사용하지 않는다.
- 질문에 대한 답변이 자연스럽게 끝나면 추가 문장은 붙이지 않는다.

---

## QA 생성 제약 (Constraints)
- **언어**: 가능하면 한국어로 질문하고 답변하되, 코드 예제는 영어로 작성하십시오.
- **문법**: 문장의 끝은 마침표(.) 등 **구두점을 반드시 표기**하여 완결된 문장으로 작성하십시오.
- **JSON 포맷 엄수**:
  - 출력은 반드시 파싱 가능한 **JSON 배열** 형태여야 합니다 (`[{"user_input": "...", "reference": "..."}, ...]`).
  - **이스케이프 처리**: 본문 내의 큰따옴표("), 백슬래시(\) 등 특수 문자는 반드시 역슬래시(\")를 사용하여 이스케이프 처리하십시오.
  - 소스 텍스트에 JSON 문법을 해치는 요소가 있더라도, 최종 출력은 유효한 JSON이어야 합니다.

---

## QA 생성 출력 예시
[
  {
    "user_input": "파인튜닝의 장점은 무엇인가?",
    "reference": "파인튜닝은 사전 학습된 모델(Foundation 모델)을 특정 태스크나 도메인 데이터로 추가 학습 최적화 하는 과정으로, 특정 작업/도메인에 최적화, 사용자 맞춤형 톤앤매너 적용, 안전성과 윤리 강화를 위해 필요하다."
  }
]

위 가이드를 바탕으로 주어진 텍스트에서 학습 가치가 높은 QA 세트를 생성하십시오.
"""

In [ ]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context=llm_context 
)

# testset
testset = generator.generate_with_chunks(
    docs, testset_size=20  # context 내용, 테스트데이터셋 몇개를 만들지.
)
testset

In [ ]:
eval_df = testset.to_pandas()
eval_df.head()

In [ ]:
# >>>> Chain 응답들을 저장할 list
response_list = []
# Chain 이 반환한 context들을 지정할 list
retrieved_context_list = []

for user_input in eval_df['user_input']:
    res = chain.invoke(user_input)
    response_list.append(res['response'])
    retrieved_context_list.append(res['retrieved_context'])

# >>>> eval_df에 응답과 context 추가
eval_df['response'] = response_list
eval_df['retrieved_contexts'] = retrieved_context_list
eval_df.head()

In [ ]:
from ragas import EvaluationDataset
# from_xxxx() xxxx 타입의 객체를 EvaluationDataset객체로 변환.
eval_dataset = EvaluationDataset.from_pandas(
    eval_df[["user_input", "retrieved_contexts", "response", "reference"]]
)
eval_dataset

In [ ]:
eval_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
# 평가할 함수들을 List로 묶어준다.
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embeddings)
]

# Run Evaluation
eval_results = evaluate(dataset=eval_dataset, metrics=metrics)

# Convert to Pandas DataFrame for easier viewing
df_results = eval_results.to_pandas()
display(df_results.head(10))

- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.

In [ ]:
print("\n📊 Evaluation Results:")
print(eval_results)

In [ ]:
eval_results.to_pandas()
eval_3rd = {'context_recall': 0.8583, 'llm_context_precision_with_reference': 0.8494, 'faithfulness': 0.8542, 'answer_relevancy': 0.8352}

print("Difference (Current - 3rd):")
for key in eval_3rd:
    diff = eval_results[key] - eval_3rd[key]
    print(f"{key}: {'+' if diff >= 0 else '-'} {diff:+.4f}")